In [2]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version="2025-03-01-preview",
    azure_endpoint = "https://aisg-sj.openai.azure.com/" # o4-mini
    # azure_endpoint = "https://decla-mbncunfi-australiaeast.cognitiveservices.azure.com/" # o3-mini
    )

In [ ]:
# Upload a file with a purpose of "batch"
file = client.files.create(
  file=open("/data/users/brandon/ob1-projects/InternVL/internvl_chat/rollout_generation/generated_rollouts/soft_estimation/RAVEN/verification/verification_pipeline_outputs/verification_batches/batch_0001.jsonl", "rb"), 
  purpose="batch",
  extra_body={"expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


print(file.model_dump_json(indent=2))

In [9]:
import datetime

print(f"File expiration: {datetime.datetime.fromtimestamp(file.expires_at) if file.expires_at is not None else 'Not set'}")

file_id = file.id

File expiration: 2025-06-25 19:32:40


In [10]:
# Submit a batch job with the file
batch_response = client.batches.create(
    input_file_id=file_id,
    endpoint="/chat/completions",
    completion_window="24h",
    extra_body={"output_expires_after":{"seconds": 1209600, "anchor": "created_at"}} # Optional you can set to a number between 1209600-2592000. This is equivalent to 14-30 days
)


# Save batch ID for later use
batch_id = batch_response.id

print(batch_response.model_dump_json(indent=2))

{
  "id": "batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7",
  "completion_window": "24h",
  "created_at": 1749641564,
  "endpoint": "/chat/completions",
  "input_file_id": "file-c9bf71e7da9d4a79a9ea47349a45e544",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1749727961,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


In [11]:
import time
import datetime 

status = "validating"
while status not in ("completed", "failed", "canceled"):
    time.sleep(60)
    batch_response = client.batches.retrieve(batch_id)
    status = batch_response.status
    print(f"{datetime.datetime.now()} Batch Id: {batch_id},  Status: {status}")

if batch_response.status == "failed":
    for error in batch_response.errors.data:  
        print(f"Error code {error.code} Message {error.message}")

2025-06-11 19:33:45.978179 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: validating
2025-06-11 19:34:46.931503 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: validating
2025-06-11 19:35:47.900976 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:36:48.886664 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:37:49.852130 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:38:50.816619 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:39:51.760855 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:40:52.752730 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:41:53.707715 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  Status: in_progress
2025-06-11 19:42:54.680854 Batch Id: batch_0d5f0af1-a2ea-4e37-8599-2ea076e1a6c7,  St

In [12]:
import json

output_file_id = batch_response.output_file_id

if not output_file_id:
    output_file_id = batch_response.error_file_id

verification_results = []
error_sample_ids = []

if output_file_id:
    file_response = client.files.content(output_file_id)
    raw_responses = file_response.text.strip().split('\n')  

    for raw_response in raw_responses:  
        json_response = json.loads(raw_response)  
        
        # Check for error status codes
        if json_response["response"]["status_code"] != 200:
            error_sample_ids.append(json_response["custom_id"])
            continue
        
        # Create new JSON format with desired schema for successful responses
        verification_entry = {
            "custom_id": json_response["custom_id"],
            "verification_response": json_response["response"]["body"]["choices"][0]["message"]["content"]
        }
        
        verification_results.append(verification_entry)
        
        # Optional: print original for debugging
        formatted_json = json.dumps(json_response, indent=2)  
        print(formatted_json)

# Save verification results to file
with open('verification_results.json', 'w') as f:
    json.dump(verification_results, f, indent=2)

print(f"\nSaved {len(verification_results)} verification results to verification_results.json")

if error_sample_ids:
    print(f"Error sample IDs with status_code != 200: {error_sample_ids}")
else:
    print("All samples processed successfully with status_code 200")

{
  "custom_id": "2904_center_single_train_79120fb1-fc17-4eb6-8771-04df0b909188",
  "response": {
    "body": {
      "choices": [
        {
          "content_filter_results": {
            "hate": {
              "filtered": false,
              "severity": "safe"
            },
            "protected_material_code": {
              "filtered": false,
              "detected": false
            },
            "protected_material_text": {
              "filtered": false,
              "detected": false
            },
            "self_harm": {
              "filtered": false,
              "severity": "safe"
            },
            "sexual": {
              "filtered": false,
              "severity": "safe"
            },
            "violence": {
              "filtered": false,
              "severity": "safe"
            }
          },
          "finish_reason": "stop",
          "index": 0,
          "logprobs": null,
          "message": {
            "annotations": [],
     

In [7]:
import json

response = client.batches.list()
response_dict = json.loads(response.model_dump_json())
active_batches = [batch for batch in response_dict["data"] if batch["status"] in ["validating", "processing"]]
print(f"Active batches: {len(active_batches)}")
print(json.dumps(active_batches, indent=2))

finalizing_batches = [batch for batch in response_dict["data"] if batch["status"] in ["finalizing"]]
print(f"Finalizing batches: {len(finalizing_batches)}")
print(json.dumps(finalizing_batches, indent=2))

# completed_batches = [batch for batch in response_dict["data"] if batch["status"] in ["completed"]]
# print(f"Completed batches: {len(completed_batches)}")
# print(json.dumps(completed_batches, indent=2))

Active batches: 0
[]
Finalizing batches: 0
[]


In [15]:
all_jobs = []
# Automatically fetches more pages as needed.
for job in client.batches.list(
    limit=20,
):
    # Do something with job here
    all_jobs.append(job)
print(all_jobs)

[Batch(id='batch_9f625636-a28e-4ad2-8d06-35cb6c884346', completion_window='24h', created_at=1749643065, endpoint='/chat/completions', input_file_id='file-f0655d2c40ee4e049038340c3e92d2cf', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id='', errors=None, expired_at=None, expires_at=1749729462, failed_at=None, finalizing_at=None, in_progress_at=1749643316, metadata=None, output_file_id='', request_counts=BatchRequestCounts(completed=212, failed=0, total=246)), Batch(id='batch_3fed68f9-03f5-48ad-b68f-65846a1f8ec0', completion_window='24h', created_at=1749641772, endpoint='/chat/completions', input_file_id='file-06149ba31d86468c94a8dc7420d2f84e', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1749643040, error_file_id=None, errors=None, expired_at=None, expires_at=1749728169, failed_at=None, finalizing_at=1749642967, in_progress_at=1749641882, metadata=None, output_file_id='file-3e2348a0-

In [ ]:
import time
from openai import BadRequestError

max_retries = 10
retries = 0
initial_delay = 5
delay = initial_delay

while True:
    try:
        batch_response = client.batches.create(
            input_file_id=file_id,
            endpoint="/chat/completions",
            completion_window="24h",
        )
        
        # Save batch ID for later use
        batch_id = batch_response.id
        
        print(f"✅ Batch created successfully after {retries} retries")
        print(batch_response.model_dump_json(indent=2))
        break  
        
    except BadRequestError as e:
        error_message = str(e)
        
        # Check if it's a token limit error
        if 'token_limit_exceeded' in error_message:
            retries += 1
            if retries >= max_retries:
                print(f"❌ Maximum retries ({max_retries}) reached. Giving up.")
                raise
            
            print(f"⏳ Token limit exceeded. Waiting {delay} seconds before retry {retries}/{max_retries}...")
            time.sleep(delay)
            
            # Exponential backoff - increase delay for next attempt
            delay *= 2
        else:
            # If it's a different error, raise it immediately
            print(f"❌ Encountered non-token limit error: {error_message}")
            raise